# Data Preprocessing

---

In this notebook, exploiting the analysis made in the previous notebook, we are going to preprocess the data to build training and testing datasets for the prediction model we need to construct to tackle the problem exposed in the current project.

For that, working on the tabular data file *train_users_2.csv*, we will proceed into various steps, following the guidelines proposed in the previous notebook:
* Gestion of the IDs
* Gestion of the ages
* Gestion of the missing values
* Gestion of the dates
* Gestion of the categorical features
* Gestion of the continuous variables

Thus, we will obtain a consistent and consolidated data file, necessary to construct the training and testing datasets.

As always, the prerequisite step consists on loading the appropriate packages to perform our work:

In [1]:
# Activate 'airbnb' environment:
!source activate airbnb

In [2]:
# Needed packages:
import numpy as np
import pandas as pd

We need, too, to load the tabular data file *train_users_2.csv*.

In [3]:
# Load the data:
train_users_2 = pd.read_csv("../data/train_users_2.csv")

# Check basic info:
print("*** Some basic info:")
print("'train_users_2' has {} data points with {} variables each.".format(*train_users_2.shape))
print("'train_users_2' counts {} missing values.".format(train_users_2.isnull().sum().sum()))

# Give a look to the first lines:
print("\n*** First lines:")
display(train_users_2.head())

*** Some basic info:
'train_users_2' has 213451 data points with 16 variables each.
'train_users_2' counts 218598 missing values.

*** First lines:


,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


---

## IDs

First, we have to make sure that each row of the dataframe concerns unique users, to prevent possible errors, and then, if it's OK, we will drop this feature.

In [4]:
# Check unique users in the dataframe:
id_unique = len(train_users_2['id'].unique().tolist())
if id_unique == train_users_2.shape[0]:
    print("It's OK: {} data points for {} unique users.".format(train_users_2.shape[0], id_unique))
else:
    print("It's KO: {} data points for {} unique users.".format(train_users_2.shape[0], id_unique))

It's OK: 213451 data points for 213451 unique users.


In [5]:
# Drop 'id':
dataset = train_users_2.drop('id', axis=1)

---

## Age

Now, we are going to manage the treatment of the `age` feature, handling the outliers, as it has been seen in the previous notebook.

In [6]:
# Handle outliers in the 'age' feature:
dataset.loc[dataset['age'] < 16, 'age'] = -1
dataset.loc[dataset['age'] > 96, 'age'] = -1

---

## Missing values

Here, we are going to treat all missing values (`NaN` values) and imprecise values (e.g. `-unknown-`).

In [7]:
# Check missing values by features:
display(dataset.isnull().sum())

date_account_created            0
timestamp_first_active          0
date_first_booking         124543
gender                          0
age                         87990
signup_method                   0
signup_flow                     0
language                        0
affiliate_channel               0
affiliate_provider              0
first_affiliate_tracked      6065
signup_app                      0
first_device_type               0
first_browser                   0
country_destination             0
dtype: int64

We have to make some choices:
* `date_first_booking` has numerous missing values: We are going to drop this feature
* `age` feature has been treated specifically above (outliers), nevertheless, we are going to handle missing values here
* `first_affiliate_tracked` will see its missing values replaced by `untracked`

In [8]:
# Drop 'date_first_booking':
dataset.drop('date_first_booking', axis=1, inplace=True)

# Handle missing values in 'age':
dataset['age'].fillna(-1, inplace=True)

# Replace in 'first_affiliate_tracked' missing values by 'untracked':
dataset['first_affiliate_tracked'].fillna('untracked', inplace=True)

# Check for missing values:
display(dataset.isnull().sum())

date_account_created       0
timestamp_first_active     0
gender                     0
age                        0
signup_method              0
signup_flow                0
language                   0
affiliate_channel          0
affiliate_provider         0
first_affiliate_tracked    0
signup_app                 0
first_device_type          0
first_browser              0
country_destination        0
dtype: int64

In [9]:
# Check for imprecise values and transformed missing values:
dataset_cols = []
for col in dataset.columns:
    dataset_cols.append(col)
dataset_cols.remove('date_account_created')
dataset_cols.remove('timestamp_first_active')
dataset_cols.remove('country_destination')
print("*** Check for imprecise values and transformed missing values:")
for col in dataset_cols:
    print("- Column '{}' possible values:\n{}".format(col, dataset[col].unique().tolist()))

*** Check for imprecise values and transformed missing values:
- Column 'gender' possible values:
['-unknown-', 'MALE', 'FEMALE', 'OTHER']
- Column 'age' possible values:
[-1.0, 38.0, 56.0, 42.0, 41.0, 46.0, 47.0, 50.0, 36.0, 37.0, 33.0, 31.0, 29.0, 30.0, 40.0, 26.0, 32.0, 35.0, 59.0, 49.0, 44.0, 34.0, 28.0, 19.0, 53.0, 52.0, 39.0, 57.0, 25.0, 54.0, 69.0, 63.0, 43.0, 55.0, 65.0, 58.0, 61.0, 18.0, 27.0, 45.0, 60.0, 48.0, 51.0, 64.0, 72.0, 70.0, 67.0, 73.0, 66.0, 68.0, 95.0, 24.0, 94.0, 75.0, 79.0, 62.0, 16.0, 23.0, 76.0, 74.0, 87.0, 92.0, 71.0, 84.0, 78.0, 82.0, 77.0, 22.0, 89.0, 21.0, 20.0, 17.0, 86.0, 81.0, 90.0, 88.0, 80.0, 91.0, 83.0, 85.0, 93.0, 96.0]
- Column 'signup_method' possible values:
['facebook', 'basic', 'google']
- Column 'signup_flow' possible values:
[0, 3, 2, 1, 24, 8, 6, 5, 10, 25, 12, 4, 16, 15, 20, 21, 23]
- Column 'language' possible values:
['en', 'fr', 'de', 'es', 'it', 'pt', 'zh', 'ko', 'ja', 'ru', 'pl', 'el', 'sv', 'nl', 'hu', 'da', 'id', 'fi', 'no', 'tr', 'th

In order to "penalize" imprecise values and transformed missing values, we are going to evaluate below the number of unknown things for a given user, and add it to the dataframe.

In [10]:
# Penalize imprecise values and transformed missing values:
dataset['nans'] = np.sum([(dataset['gender']=='-unknown-'),
                          (dataset['age']==-1),
                          (dataset['first_affiliate_tracked']=='untracked'),
                          (dataset['first_browser']=='-unknown-')
                         ], axis=0)

---

## Dates

Here, we are going to transform the 2 remaining dates, `date_account_created` and `timestamp_first_active`, and extract from them the valuable information.

In [11]:
# Cast dates to proper format:
dataset['date_account_created'] = pd.to_datetime(dataset['date_account_created'])
dataset['date_first_active'] = pd.to_datetime(dataset['timestamp_first_active'], format='%Y%m%d%H%M%S')

# Convert dates to 'DateTime Index':
date_account_created = pd.DatetimeIndex(dataset['date_account_created'])
date_first_active = pd.DatetimeIndex(dataset['date_first_active'])

# Split dates into day, week, month and year:
dataset['day_account_created'] = date_account_created.day
dataset['weekday_account_created'] = date_account_created.weekday
dataset['week_account_created'] = date_account_created.week
dataset['month_account_created'] = date_account_created.month
dataset['year_account_created'] = date_account_created.year
dataset['day_first_active'] = date_first_active.day
dataset['weekday_first_active'] = date_first_active.weekday
dataset['week_first_active'] = date_first_active.week
dataset['month_first_active'] = date_first_active.month
dataset['year_first_active'] = date_first_active.year

# Calculate time lag between first activity date and account creation date:
dataset['time_lag'] = (date_account_created.values - date_first_active.values).astype(int)

# Drop the now useless variables from the data set:
fields_to_drop = ['date_account_created',
                  'date_first_active',
                  'timestamp_first_active']
dataset.drop(fields_to_drop, axis=1, inplace=True)

---

## Categorical features

It's now time to handle the categorical variables of our dataset, to include them in our model, and for that, we will need to make binary dummy variables.

In [12]:
# List of fields to dummify:
dummy_fields = ['gender',
                'signup_method',
                'signup_flow',
                'language',
                'affiliate_channel',
                'affiliate_provider',
                'first_affiliate_tracked',
                'signup_app',
                'first_device_type',
                'first_browser']

# Create dummy variables and add them to the data set:
for each in dummy_fields:
    dummies = pd.get_dummies(dataset[each], prefix=each, drop_first=False)
    dataset = pd.concat([dataset, dummies], axis=1)

# Drop now useless variables from the data set:
dataset.drop(dummy_fields, axis=1, inplace=True)

---

## Continuous variables

Finally, we are going to standardize each of the continuous variables, that is, we will shift and scale the variables such that they have 0 mean and a standard deviation of 1.

*Nota Bene:* Due to the fact that we have introduced the value `-1` for `NaN` values and outliers for the `age` feature, we are not going to treat it here.

In [13]:
# Determine the continuous variables to standardize:
continuous_features = ['nans', 'time_lag']

# Store scalings in a dictionary to convert back later and standardize:
scaled_features = {}
for each in continuous_features:
    mean, std = dataset[each].mean(), dataset[each].std()
    scaled_features[each] = [mean, std]
    dataset.loc[:, each] = (dataset[each] - mean)/std

---

## Save our work

Our dataset is now consolidated: Before going further, we can save it.

In [14]:
dataset.to_csv("../data/consolidated_dataset.csv", header=True, index=False)